<a href="https://colab.research.google.com/github/RainbowPowerr/ML-thesis/blob/main/Stable_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Stable diffusion**

Set up:

In [1]:
!nvidia-smi

!pip install diffusers==0.4.0
!pip install transformers scipy ftfy
!pip install "ipywidgets>=7,<8"

from huggingface_hub import notebook_login

notebook_login()

/bin/sh: line 1: nvidia-smi: command not found


ModuleNotFoundError: No module named 'google'

Download model and prep grid

In [2]:
from diffusers import StableDiffusionPipeline
import torch

model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16, revision="fp16")
pipe = pipe.to("cuda")

from PIL import Image

def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid
all_images = []

import os

def createFolder(directory):
    try:
        if not os.path.exists(directory):
            os.makedirs(directory)
    except OSError:
        print ('Error: Creating directory. ' +  directory)

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

Generating single images (with and without grid)

In [ ]:
def image_grid(imgs, rows, cols):
    assert len(imgs) == rows*cols

    w, h = imgs[0].size
    grid = Image.new('RGB', size=(cols*w, rows*h))
    grid_w, grid_h = grid.size
    
    for i, img in enumerate(imgs):
        grid.paste(img, box=(i%cols*w, i//cols*h))
    return grid
all_images = []

num_cols = 3
num_rows = 4

prompt = ["/content/forest.jpg, a forest in Sweden by a lake, autumn, 4k, hd"] * num_cols
generator = torch.Generator("cuda").manual_seed(984799)


for i in range(num_rows):
  images = pipe(prompt, num_inference_steps=50, guidance_scale = 8, generator=generator).images
  all_images.extend(images)

grid = image_grid(all_images, rows=num_rows, cols=num_cols)
grid


In [11]:
import os
print(os.getcwd())
generator = torch.Generator("cuda").manual_seed(1234)
prompt = "./Images/Guide_Images/coinstack.jpg, straight view of a tiny red house next to a stack of golden coins, 4k, hd"
image = pipe(prompt, num_inference_steps=50, guidance_scale = 20, generator=generator, height=400 , width=720 ).images[0]

image.save("./Images/Test_Images/Housing_market.png")

/home/stablediffusion/Documents/GitHub/ML-thesis


  0%|          | 0/51 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
generator = torch.Generator("cuda").manual_seed(1234)
prompt = "/content/forest.jpg, a forest in Sweden by a lake, autumn, sun is shining, 4k, hd"
image = pipe(prompt, num_inference_steps=50, guidance_scale = 20, generator=generator, height=400 , width=720 ).images[0]

image.save("/content/Test_images/forest.png")

In [ ]:
generator = torch.Generator("cuda").manual_seed(12345)
prompt = "/content/boxes.jpg, cardboard boxes and plants, on a table, in an office, folders, sideview, 4k, hd"
image = pipe(prompt, num_inference_steps=50, guidance_scale = 20, generator=generator, height=400 , width=720 ).images[0]

image.save("/content/Test_images/bankruptcy.png")

# Doing the thing
REMEMBER TO CHANGE THE NUMBER OF PIXELS WHEN DOING FOR REAL

In [18]:
import pandas as pd
project_root = "./Images"
df = pd.read_csv(r'./image_data.csv')
print(len(df))
print(df.head())

1000
   Unnamed: 0      prompt  guidance  inf_steps  xtra_prompt  guide_image_bin  \
0           1    houseing       4.0         71            1                0   
1           2     weather       3.4        166            1                0   
2           3     weather      11.7         58            0                1   
3           4  bankruptcy       6.6        179            1                0   
4           5     weather       1.6         64            0                0   

    image  
0    none  
1    none  
2  forest  
3    none  
4    none  


In [19]:
def extra(xtra_prompt):
  if xtra_prompt == 1:  
    extra_words = ", cinematic, colorful background, high detail, highly detailed, hyper realistic, intricate, intricate sharp details, smooth, studio lighting"
  else: 
    extra_words = " "
  return extra_words

def guide(guide_image_bin, image):
  if guide_image_bin == 1 and image == "coinstack": 
    guide_image = f"{project_root}/Guide_Images/coinstack.jpg,"
  elif guide_image_bin == 1 and image == "boxes": 
    guide_image = f"{project_root}/Guide_Images/boxes.jpg,"
  elif guide_image_bin == 1 and image == "forest": 
    guide_image = f"{project_root}/Guide_Images/forest.jpg,"
  else: guide_image = " "
  return guide_image

def main(prompt):
  if prompt == "houseing": main_prompt =  " straight view of a tiny red house next to a stack of golden coins, 4k, hd"
  elif prompt == "weather": main_prompt =  " a forest in Sweden by a lake, autumn, sun is shining, 4k, hd"
  elif prompt == "bankruptcy": main_prompt =  " cardboard boxes and plants, on a table, in an office, folders, sideview, 4k, hd"
  else: main_prompt = " "
  return main_prompt

In [21]:
# Loop för att spara flera bilder i mappen Test_images

createFolder('./Images/Main_Images/')

num_images = len(df)

for i in range(num_images):
  generator = torch.Generator("cuda").manual_seed(6596 + i)
  with open("log.txt", "w") as f:
    f.write(f"Iteration {i}")
  prompt = df["prompt"][i]
  image = df["image"][i]
  guide_image_bin = df["guide_image_bin"][i]
  xtra_prompt = df["xtra_prompt"][i]
  images = pipe(prompt=(guide(guide_image_bin, image)+main(prompt)+extra(xtra_prompt)), num_inference_steps=df["inf_steps"][i], guidance_scale = df["guidance"][i], height=400 , width=720, generator=generator ).images
  images[0].save(f'./Images/Main_Images/{i}.png')

  0%|          | 0/72 [00:00<?, ?it/s]

  0%|          | 0/167 [00:00<?, ?it/s]

  0%|          | 0/59 [00:00<?, ?it/s]

  0%|          | 0/180 [00:00<?, ?it/s]

  0%|          | 0/65 [00:00<?, ?it/s]

...

  0%|          | 0/67 [00:00<?, ?it/s]

  0%|          | 0/57 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

  0%|          | 0/93 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
!zip -r /content/Images.zip /content/Images